# Preliminary work
The session showcases how to infer activities of transcription factor from single cell RNA-seq data and spatial transcriptomics data using three method. Please follow this notebook after you have [set up the environment](https://github.com/osmanbeyoglulab/Tutorials-on-ISMB-2024?tab=readme-ov-file#environment-set-up).

In [1]:
import scanpy as sc
import warnings
warnings.filterwarnings("ignore")

## Downloading datasets
The following datasets will be used for demonstration, which can be downloaded via `scanpy` to the directory `data`.

In [2]:
sc.datasets.pbmc3k()

AnnData object with n_obs × n_vars = 2700 × 32738
    var: 'gene_ids'

In [3]:
sc.datasets.pbmc3k_processed()

AnnData object with n_obs × n_vars = 2638 × 1838
    obs: 'n_genes', 'percent_mito', 'n_counts', 'louvain'
    var: 'n_cells'
    uns: 'draw_graph', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_tsne', 'X_umap', 'X_draw_graph_fr'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [4]:
sc.datasets.visium_sge(sample_id="V1_Human_Lymph_Node")

AnnData object with n_obs × n_vars = 4035 × 36601
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [5]:
sc.datasets.visium_sge(sample_id="Parent_Visium_Human_Glioblastoma")

AnnData object with n_obs × n_vars = 3468 × 36601
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

## decoupleR
[decoupler](https://doi.org/10.1093/bioadv/vbac016) is a package containing different statistical methods to extract biological activities from omics data within a unified framework, including pathway activity inference and transcription factor activity inference. We follow [the instruction](https://decoupler-py.readthedocs.io/en/latest/installation.html) to install decoupler.

In [6]:
pip install decoupler

Note: you may need to restart the kernel to use updated packages.


In [7]:
import decoupler
decoupler.__version__

'1.5.0'

## pySCENIC (optional)
[pySCENIC](https://www.nature.com/articles/s41596-020-0336-2) is a package containing different statistical methods to extract biological activities from single-cell RNA-seq data within a unified framework, including gene regulatory network inference and transcription factor activity inference. We follow [the instruction](https://pyscenic.readthedocs.io/en/latest/installation.html) to install pySCENIC. pySCENIC depends on packages e.g. arboreto and ctxcore. **To install and use pySCENIC, we recommend to create a new conda envorinment as instructed.**

In [ ]:
pip install pyscenic

In [ ]:
import pyscenic
pyscenic.__version__

### Downloading resources and databases

In [ ]:
!mkdir resources_pyscenic
!curl https://resources.aertslab.org/cistarget/tf_lists/allTFs_hg38.txt \
    -o resources_pyscenic/allTFs_hg38.txt
!curl https://resources.aertslab.org/cistarget/motif2tf/motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl \
    -o resources_pyscenic/motifs-v10nr_clust-nr.hgnc-m0.001-o0.0.tbl
!curl https://resources.aertslab.org/cistarget/databases/homo_sapiens/hg38/refseq_r80/mc_v10_clust/gene_based/hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather \
    -o resources_pyscenic/hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather
!curl https://resources.aertslab.org/cistarget/databases/homo_sapiens/hg38/refseq_r80/mc_v10_clust/gene_based/hg38_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather \
    -o resources_pyscenic/hg38_500bp_up_100bp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather

## STAN
[STAN](https://www.biorxiv.org/content/10.1101/2024.06.26.600782v1) is a computational framework for inferring spatially informed transcription factor activity across cellular contexts. Specifically, STAN is a linear mixed-effects computational method that predicts spot-specific, spatially informed TF activities by integrating curated TF-target gene priors, mRNA expression, spatial coordinates, and morphological features from corresponding imaging data. We install additional packages to support STAN.

In [8]:
pip install squidpy==1.5.0

ERROR: Ignored the following versions that require a different python version: 1.3.0 Requires-Python >=3.9; 1.3.1 Requires-Python >=3.9; 1.4.0 Requires-Python >=3.9; 1.4.1 Requires-Python >=3.9; 1.5.0 Requires-Python >=3.9
ERROR: Could not find a version that satisfies the requirement squidpy==1.5.0 (from versions: 1.0.0, 1.0.1, 1.1.0, 1.1.1, 1.1.2, 1.2.0, 1.2.1, 1.2.2, 1.2.3)
ERROR: No matching distribution found for squidpy==1.5.0
Note: you may need to restart the kernel to use updated packages.


In [9]:
import squidpy
squidpy.__version__

'1.2.3'

In [10]:
pip install statsmodels==0.14.2

ERROR: Ignored the following versions that require a different python version: 0.14.2 Requires-Python >=3.9
ERROR: Could not find a version that satisfies the requirement statsmodels==0.14.2 (from versions: 0.4.0, 0.4.1, 0.4.3, 0.5.0rc1, 0.5.0, 0.6.0rc1, 0.6.0rc2, 0.6.0, 0.6.1, 0.8.0rc1, 0.8.0, 0.9.0rc1, 0.9.0, 0.10.0rc2, 0.10.0, 0.10.1, 0.10.2, 0.11.0rc1, 0.11.0rc2, 0.11.0, 0.11.1, 0.12.0rc0, 0.12.0, 0.12.1, 0.12.2, 0.13.0rc0, 0.13.0, 0.13.1, 0.13.2, 0.13.3, 0.13.4, 0.13.5, 0.14.0rc0, 0.14.0, 0.14.1)
ERROR: No matching distribution found for statsmodels==0.14.2
Note: you may need to restart the kernel to use updated packages.


In [11]:
import statsmodels
statsmodels.__version__

'0.14.0'

### Downloading supporting files

In [12]:
!mkdir resources_stan
!curl https://raw.githubusercontent.com/vitkl/cell2location_paper/1c645a0519f8f27ecef18468cf339d35d99f42e7/notebooks/selected_results/lymph_nodes_analysis/CoLocationModelNB4V2_34clusters_4039locations_10241genes_input_inferred_V4_batch1024_l2_0001_n_comb50_5_cps5_fpc3_alpha001/W_cell_density.csv \
    -o resources_stan/W_cell_density.csv
!curl https://raw.githubusercontent.com/vitkl/cell2location_paper/1c645a0519f8f27ecef18468cf339d35d99f42e7/notebooks/selected_results/lymph_nodes_analysis/CoLocationModelNB4V2_34clusters_4039locations_10241genes_input_inferred_V4_batch1024_l2_0001_n_comb50_5_cps5_fpc3_alpha001/manual_GC_annot.csv \
    -o resources_stan/manual_GC_annot.csv

mkdir: resources_stan: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2660k  100 2660k    0     0  38788      0  0:01:10  0:01:10 --:--:-- 3631329k    0     0  43258      0  0:01:02  0:00:17  0:00:45 25417   0  0:01:05  0:00:20  0:00:45 26574:21  0:00:45 270865 278250:00:51  0:00:20 33676:01:11  0:00:56  0:00:15 37506  0  0:01:11  0:01:02  0:00:09 42270
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 81467  100 81467    0     0   103k      0 --:--:-- --:--:-- --:--:--  104k


In [13]:
import session_info
session_info.show()